In [1]:
!pip install urduhack
!pip install tensorflow

In [3]:
import urduhack
urduhack.download()

2024-06-05 16:56:36.538390: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-05 16:56:36.538449: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-05 16:56:36.539996: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our Tens

89078/89078 [==============================] - 0s 0us/step


In [4]:
import pandas as pd
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM, Embedding, Reshape, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# Specify the path to your Excel file


In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense



# TextGAN

In [51]:
#libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape, LSTM, Flatten
# using textgan
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Reshape, Flatten
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.preprocessing.sequence import pad_sequences
import urduhack
from urduhack.preprocessing import normalize_whitespace, remove_punctuation
from urduhack.tokenization import word_tokenizer
from urduhack.normalization import normalize


In [52]:
# Assuming you have functions normalize, normalize_whitespace, remove_punctuation, word_tokenizer defined
def preprocess_text(text):
    text = normalize(text)
    text = normalize_whitespace(text)
    text = remove_punctuation(text)
    tokens = word_tokenizer(text)
    return tokens


In [68]:
# data loading 
dataset_path = "/kaggle/input/urdu-news-csv/news.csv"

df = pd.read_csv(dataset_path, encoding='utf-8')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
org_data = df.copy()
print(df.shape)
df.head()

(4097, 2)


,news,label
0,\tپاکستان کے صوبے پنجاب میں گذشتہ ہفتے ہونے وا...,real
1,\tوزیر اعظم عمران خان کے ساجد گوندل کی بازیابی...,real
2,لاہور(کلچرل رپورٹر)ایشین ایکسی لینس پرفارمنس ا...,fake
3,امریکی خواتین سوکر اسٹار میگن ریپینو گھٹنوں کے...,fake
4,\tوفاق ہو یا صوبے، خیبر پختونخواہ ہو یا پنجاب،...,real


In [69]:
# Pre Processing using urduhack functions
df = df.head(10)  # Limit to the top 10 rows due to resource limitations you can increase
df['label'] = df['label'].str.strip()
df['tokens'] = df['news'].apply(preprocess_text)


9/9 [==============================] - 1s 112ms/step


In [70]:
df.head() # latest data set wit tokens 

,news,label,tokens
0,\tپاکستان کے صوبے پنجاب میں گذشتہ ہفتے ہونے وا...,real,"[پاکستان, کے, صوبے, پنجاب, میں, گذشتہ, ہفتے, ہ..."
1,\tوزیر اعظم عمران خان کے ساجد گوندل کی بازیابی...,real,"[وزیراعظم, عمران, خان, کے, ساجد, گوندل, کی, با..."
2,لاہور(کلچرل رپورٹر)ایشین ایکسی لینس پرفارمنس ا...,fake,"[لاہور, کلچرل, رپورٹر, ایشین, ایکسیلینس, پرفار..."
3,امریکی خواتین سوکر اسٹار میگن ریپینو گھٹنوں کے...,fake,"[امریکی, خواتین, سو, کر, اسٹار, میگن, ریپینو, ..."
4,\tوفاق ہو یا صوبے، خیبر پختونخواہ ہو یا پنجاب،...,real,"[وفاق, ہو, یا, صوبے, خیبر, پختونخواہ, ہو, یا, ..."


In [71]:
# tokeniazation , pad sequencing 
all_tokens = [token for sublist in df['tokens'] for token in sublist]
vocab = list(set(all_tokens))
token_to_id = {token: idx for idx, token in enumerate(vocab)}
df['token_ids'] = df['tokens'].apply(lambda tokens: [token_to_id[token] for token in tokens])
max_sequence_length = max([len(token_ids) for token_ids in df['token_ids']])
padded_sequences = pad_sequences(df['token_ids'], maxlen=max_sequence_length, padding='post')


In [72]:
# Preparing data for  GAN 
data = pd.DataFrame(padded_sequences)
data['label'] = df['label'].apply(lambda x: 1 if x == 'real' else 0)
data['label'] = data['label'].astype(int)

for column in data.columns:
    if column != 'label':
        data[column] = data[column].astype(float)

data.columns = data.columns.astype(str)
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)
data_tensor = tf.convert_to_tensor(data_scaled, dtype=tf.float32)
label_tensor = tf.convert_to_tensor(data['label'].values, dtype=tf.float32)
dataset = tf.data.Dataset.from_tensor_slices((data_tensor, label_tensor)).shuffle(buffer_size=1024).batch(32)


In [73]:
# defoinning generator and discriminator yuo can enhance its complexity accoridng to need 
latent_dim = 100
output_dim = data_tensor.shape[1]

generator = Sequential([
    Dense(128, input_dim=latent_dim, activation='relu'),
    Reshape((1, 128)),
    LSTM(256, return_sequences=True),
    Flatten(),
    Dense(output_dim, activation='sigmoid')
])
input_dim = data_tensor.shape[1]

discriminator = Sequential([
    Reshape((1, input_dim), input_shape=(input_dim,)),
    LSTM(256, return_sequences=True),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])


In [74]:
# Defining TextGAN
optimizer = tf.keras.optimizers.legacy.Adam(lr=0.0002)
discriminator.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
discriminator.trainable = False

gan_input = tf.keras.Input(shape=(latent_dim,))
generated_data = generator(gan_input)
gan_output = discriminator(generated_data)
gan = tf.keras.Model(gan_input, gan_output)
gan.compile(optimizer=optimizer, loss='binary_crossentropy')



/opt/conda/lib/python3.10/site-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [75]:
# training GAN
epochs = 100

for epoch in range(epochs):
    for real_data, _ in dataset:
        batch_size = real_data.shape[0]

        # Train Discriminator
        real_labels = tf.ones((batch_size, 1))
        fake_labels = tf.zeros((batch_size, 1))

        z = tf.random.normal((batch_size, latent_dim))
        fake_data = generator(z)

        d_loss_real = discriminator.train_on_batch(real_data, real_labels)
        d_loss_fake = discriminator.train_on_batch(fake_data, fake_labels)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train Generator
        z = tf.random.normal((batch_size, latent_dim))
        g_loss = gan.train_on_batch(z, real_labels)

    print(f'Epoch [{epoch+1}/{epochs}], d_loss: {d_loss[0]:.4f}, g_loss: {g_loss:.4f}')


Epoch [1/100], d_loss: 0.6969, g_loss: 0.7395
Epoch [2/100], d_loss: 0.6746, g_loss: 0.7631
Epoch [3/100], d_loss: 0.6576, g_loss: 0.7884
Epoch [4/100], d_loss: 0.6416, g_loss: 0.8116
Epoch [5/100], d_loss: 0.6269, g_loss: 0.8368
Epoch [6/100], d_loss: 0.6118, g_loss: 0.8634
Epoch [7/100], d_loss: 0.5967, g_loss: 0.8907
Epoch [8/100], d_loss: 0.5812, g_loss: 0.9204
Epoch [9/100], d_loss: 0.5667, g_loss: 0.9502
Epoch [10/100], d_loss: 0.5523, g_loss: 0.9805
Epoch [11/100], d_loss: 0.5385, g_loss: 1.0099
Epoch [12/100], d_loss: 0.5246, g_loss: 1.0415
Epoch [13/100], d_loss: 0.5114, g_loss: 1.0735
Epoch [14/100], d_loss: 0.4983, g_loss: 1.1071
Epoch [15/100], d_loss: 0.4854, g_loss: 1.1394
Epoch [16/100], d_loss: 0.4731, g_loss: 1.1727
Epoch [17/100], d_loss: 0.4601, g_loss: 1.2082
Epoch [18/100], d_loss: 0.4486, g_loss: 1.2479
Epoch [19/100], d_loss: 0.4350, g_loss: 1.2820
Epoch [20/100], d_loss: 0.4235, g_loss: 1.3232
Epoch [21/100], d_loss: 0.4118, g_loss: 1.3624
Epoch [22/100], d_loss

In [76]:
# generating text 
num_samples = 100

z = tf.random.normal((num_samples, latent_dim))
synthetic_data = generator(z).numpy()
synthetic_data = scaler.inverse_transform(synthetic_data)
print(synthetic_data)

[[200.91508    190.78592    191.20383    ...  28.493881    24.403746
    0.4041466 ]
 [200.71748    190.56122    190.99663    ...  28.342272    24.311543
    0.40273866]
 [200.6628     190.46378    190.89633    ...  28.231369    24.259296
    0.40232685]
 ...
 [200.87305    190.70541    191.08755    ...  28.52375     24.375051
    0.4035004 ]
 [200.94673    190.74449    191.2077     ...  28.548962    24.40829
    0.40408596]
 [201.84502    191.88191    192.02759    ...  29.314667    24.81385
    0.40905926]]


# Text Geenration 

In [77]:
# converting generated text into back  in urdu 
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Assuming 'dataset' is already created from the code provided
data_tensor, label_tensor = zip(*[(data, label) for data, label in dataset])
data_array = tf.concat(data_tensor, axis=0).numpy()
label_array = tf.concat(label_tensor, axis=0).numpy()

# Convert back to DataFrame
data_df = pd.DataFrame(data_array)
data_df['label'] = label_array

# Separate features and labels
features = data_df.drop(columns='label')
labels = data_df['label']

# Assuming 'scaler' is the fitted StandardScaler from the provided code
# Apply inverse transform only to the feature columns
features_original = scaler.inverse_transform(features)

# Create a DataFrame for the original data
data_df = pd.DataFrame(features_original, columns=features.columns)
data_df['label'] = labels

# Reconstruct the token_to_id dictionary
all_tokens = [token for sublist in df['tokens'] for token in sublist]
vocab = list(set(all_tokens))
token_to_id = {token: idx for idx, token in enumerate(vocab)}
id_to_token = {idx: token for token, idx in token_to_id.items()}

# Remove padding
def remove_padding(sequence, pad_value=0):
    return [int(token_id) for token_id in sequence if token_id != pad_value]

synthetic_sequences = [remove_padding(seq) for seq in data_df.drop(columns='label').values]

# Convert token IDs back to tokens
def ids_to_tokens(ids, id_to_token):
    return [id_to_token[int(id)] for id in ids if int(id) in id_to_token]

synthetic_texts = [ids_to_tokens(seq, id_to_token) for seq in synthetic_sequences]

# Join tokens to form text
def tokens_to_text(tokens):
    return ' '.join(tokens)

synthetic_texts = [tokens_to_text(tokens) for tokens in synthetic_texts]

# Display the generated texts
for i, text in enumerate(synthetic_texts):
    print(f"Synthetic Text {i+1}: {text}\n")


Synthetic Text 1: سوسائٹیسٹ کو پیدا سمجھتے والے پیسے اور رقم چیمپلین جزیروں میں سمر ہوم کے نئے مالک سیون لdidیہ نے تصدیق کیہے برلنگٹن کارہائشی گذشتہ برائیوں نارتھ ہیروکے ایک جھیل فرنٹ کیمپ اے کر$600000 گرپڑاسینڈرس کی نئی پالنے میں چاربیڈ روم اورمشرق اور طرف جھیل چیمپلن ساحل سمندرکے 500 فٹہے جزیرے کا سامنا نیویارک کے بجا ئے ورمونٹکاہے برنا سمجھتے سمجھتے سمجھتے سمجھتے سمجھتے سمجھتے سمجھتے سمجھتے سمجھتے سمجھتے سمجھتے سمجھتے سمجھتے سمجھتے سمجھتے

Synthetic Text 2: مڈ ویڈنگ کے بارے میں معلوم کرنے کے بعد کہ اس کا جوڑا ٹھیک نہیں طرف دلہن نے اس کے بجا ئے شادی کے مہم ان سے شادی کی سمجھتے سمجھتے سمجھتے سمجھتے سمجھتے سمجھتے سمجھتے سمجھتے سمجھتے سمجھتے سمجھتے سمجھتے سمجھتے سمجھتے سمجھتے سمجھتے سمجھتے سمجھتے سمجھتے سمجھتے سمجھتے سمجھتے سمجھتے سمجھتے سمجھتے سمجھتے سمجھتے سمجھتے سمجھتے سمجھتے سمجھتے سمجھتے سمجھتے سمجھتے سمجھتے سمجھتے

Synthetic Text 3: وفاق ہو یا صوبے خیبر پختونخواہ ہو یا پنجاب بزدار صاحب ہیںکیوںکہ اور ای سی اعلی مثال بن گئے ہیںجن کا ذکرآپ کوگورنرجنرل غلام محمدکی طرح صرف تاریخ کی کتا

In [108]:
import pandas as pd

# Assuming synthetic_texts and synthetic_labels are already created
# Create a DataFrame for the synthetic texts and labels
synthetic_data_df = pd.DataFrame({
    'news': synthetic_texts,
    'label': synthetic_labels
})

# Combine the synthetic DataFrame with the original DataFrame
combined_df = pd.concat([org_data, synthetic_data_df], ignore_index=True)

# Display the combined DataFrame
print(combined_df.shape)
combined_df.head()


(4107, 2)


,news,label
0,\tپاکستان کے صوبے پنجاب میں گذشتہ ہفتے ہونے وا...,real
1,\tوزیر اعظم عمران خان کے ساجد گوندل کی بازیابی...,real
2,لاہور(کلچرل رپورٹر)ایشین ایکسی لینس پرفارمنس ا...,fake
3,امریکی خواتین سوکر اسٹار میگن ریپینو گھٹنوں کے...,fake
4,\tوفاق ہو یا صوبے، خیبر پختونخواہ ہو یا پنجاب،...,real


> as you can see the org data shap was (4097,2) and we have generated 10 rows new so adding them in org data it becomes (4107,3)

# Applying Model 

> firstly applying adaboost so we have trioi convert combined_df back to tokenize and then we will apply adaboost

In [ ]:
import pandas as pd
from urduhack.preprocessing import normalize_whitespace, remove_punctuation
from urduhack.tokenization import word_tokenizer
from urduhack.normalization import normalize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report, accuracy_score


# Assuming combined_df already contains your data
print("Total number of rows in the dataset:", len(combined_df))

# Define the preprocess_text function
def preprocess_text(text):
    text = normalize(text)
    text = normalize_whitespace(text)
    text = remove_punctuation(text)
    tokens = word_tokenizer(text)
    return tokens

# Apply the preprocessing function to the dataframe
combined_df['tokens'] = combined_df['news'].apply(preprocess_text)

# Remove Urdu stop words
urdu_stop_words = ["کے", "میں", "کی", "کا", "ہے", "کو", "سے", "پر", "اور"]  # Example stop words

def remove_stop_words(tokens):
    filtered_tokens = [token for token in tokens if token not in urdu_stop_words]
    return ' '.join(filtered_tokens)

combined_df['cleaned_news'] = combined_df['tokens'].apply(remove_stop_words)

# Feature extraction using TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(combined_df['cleaned_news'])

# Label encoding
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(combined_df['label'])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# AdaBoost implementation
model = AdaBoostClassifier(n_estimators=50, random_state=42)
model.fit(X_train, y_train)

# Predictions and evaluation
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_, zero_division=0))

# Verify the number of rows processed
print("Number of rows processed:", len(combined_df))


Total number of rows in the dataset: 4107
62/62 [==============================] - 7s 111ms/step
